In [1]:
%gui qt
%pylab qt
import numpy as np
import re
from pyknotid.spacecurves import Knot
from pyknotid.representations import GaussCode, Representation
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from tqdm import tnrange, tqdm_notebook
from time import sleep

Populating the interactive namespace from numpy and matplotlib


### Splitter 
__Overview:__ The splitter() function parses PyKnotId gauss code in preparation for conversion.

In [2]:
def splitter(s):
    """
    input:
        s - a string triplet 
    
    output:
        number - the numerical character of the triplet
        sign   - the sign character of the triplet
        hand   - the clockwise (or anticlockwise) character of the triplet
    """
    
    if '+' in s:                    # If 's' contains the '+' character, 
        
        sign = '+'                     # Then set sign = '+'
            
    elif '-' in s:                  # However, if 's' contains the '-' character,
        
        sign = '-'                     # Then set sign = '-'
    
    number, hand = s.split(sign)    # set number equal to the characters before 'sign' and hand equal to the characters after 'sign'

        
    return number, sign, hand            

### Gauss Converter
__Overview:__ This function translates Knot_Awesome gauss code into PyKnotId gauss code

In [1]:

def gauss_converter(g,e):
    """
    input:
        g - a list of integers representing the gauss code of a knot
        e - a list of integers represeing the extension to the knot's gauss code
        
    output:
        a PyKnotId gauss code
    """
    
    newgauss = ''
    
    for i in g: # Iterate through each element of the gauss code,'g'
    
        if i < 0:                             # if the current element is negative
        
            # c for clockwise
            # a for anticlockwise
            
            if e[abs(i)-1]==1:                    # and if the crossing type is clockwise
            
                newgauss += str(abs(i))+'-c,'         # then add the string '-c,' to newgauss
            
            else:                                 # however, if the crossing type is anticlockwise
                
                newgauss += str(abs(i))+'-a,'         # then add the string '-a,' to newgauss
        
        else:                                 # however, if the current element is non-negative
        
            if e[abs(i)-1]==1:                    # and if the crossing type is clockwise
            
                newgauss += str(abs(i))+'+c,'         # then add the string '+c,' to newgauss
            
            else:                                 # however, if the crossing type is anticlockwise
                
                newgauss += str(abs(i))+'+a,'         # then add the string '+a,' to newgauss
    
    return newgauss[:-1]



### Gauss Reverter
__Overview:__ Converts PyKnotId gauss code into Knot_Awesome gauss code

In [4]:
def gauss_reverter(g):
    """
    input:
        g - a list of strings in PyKnotId gauss code format
    
    output:
        numbers   - a list of intgers in Knot_Awesome gauss code format
        ex_g_code - a list of integers representing the gauss code extension in Knot_Awesome format
    """
    
    # define empty lists to keep track of each element of the gauss code
    numbers = []
    signs = []
    hands = []
     
    gauss = g.split(',') # split g into a list using the ',' character as the separating mark
    
    for i in gauss: # iterate through each element of the PyKnotId gauss code
    
        number, sign, hand = splitter(i) # split each element into its number, sign, and hand components
        numbers.append(int(number))      # append the 'number' component to 'numbers'
        signs.append(sign)               # append the 'sign' component to 'signs'
        hands.append(hand)               # append the 'hand' component to 'hands'
            
    # define a list of zeros that will eventually become the extension of the gauss code
    ex_g_code = list(np.zeros(max(numbers),dtype=int)) # this list is the length of the number of crossings in the knot

    for i,j in enumerate(numbers): # Iterate through 'numbers'
        # in this loop, j represents the crossing number
        
        if signs[i] == '-':               # if the i-th sign is negative
        
            numbers[i] *= -1                  # then we have an undercrossing, so make the number negative
        
        if ex_g_code[abs(j)-1] == 0:      # if the j-th entry of the extended gauss code has not been updated
            
            if hands[i] == 'a':               # and if the orientation of crossing j is anticlockwise
            
                ex_g_code[abs(j)-1] = -1          # then the extended gauss code gets a -1 in the j-th index
           
            else:                             # however if the orientation of crossing j is clockwise
            
                ex_g_code[abs(j)-1] = 1           # then the extended gauss code gets a 1 in the j-th index
                

    return numbers, ex_g_code

### Get the Gauss Codes
__Overview:__ The get_code() function generates the gauss code for a randomly generated knot.

In [5]:
def get_code(low=10,high=50,run=20,reparam=100000,interp_pts=400,pt_random = True,point_amt=0):
    """
    input:
        low        - integer a lower bound on the number of crossings the knot should have
        high       - integer an upper bound on the number of crossings the knot should have
        run        - the number of simplifications to run on the knot
        reparam    - integer representing the number of parameters to describe the knot
        interp_pts - the number of points to interpolate the b-spline through
        pt_random  - boolean value indicating whether to randomly sample points on a unit sphere or not
        point_amt  - number of points to sample
    
    output:
        total_gauss - the resulting gauss code from the randomly generated knot
    """
    
    pm = lambda x:x*np.random.choice([-1,1]) # This lambda function samples a point on a uniform distribution
    
    if pt_random: # if the user specified random points are to be used
    
        point_amt = np.random.randint(low,high) # Then get a random integer on the interval [low, high] indicating the number of points to sample
    
    mult = np.array([np.random.randint(1,30) for i in range(point_amt)]) # get an array of length 'point_amt' of random points
    points = np.random.rand(point_amt,3) # get a random list of length 'point_amt' points in 3-space
    
    for i,j in enumerate(points): # Iterate through each point
        
        if j[0]==j[1] and j[1]==j[2] and j[2] == 0: # Exclude the possibility of including the origin
        
            break
        
        # The following three lines of code normalize the point, placeing them on a unit sphere.
        points[i] = [pm(j) for j in points[i]]
        points[i] *= 1/np.sqrt(j[0]**2+j[1]**2+j[2]**2)
        points[i]*= point_amt

        
    gc=Knot(points).gauss_code() #Generate knot object from points tuple
    
    gcs = gc.strang() # use the strang() function to change the gauss code from a list into a string

    if len(gc)>2: # ensure the knot has at least 3 crossings 
        
        gauss1, extend1 = gauss_reverter(gcs) # convert the gauss code
        gc.simplify() # simplify the gauss code
        
    else: # in the event the knot has less than 3 crossings, it must be the unknot
        
        gauss1 = [0]
        extend1 = [0]
        
    if len(gc)>2: # ensure the knot has at least 3 crossings
        
        gcs = gc.strang() # convert the gauss code, but do not simplify
        gauss2, extend2 = gauss_reverter(gcs)
        
    else: # if the knot has less than 3 crossings, it must be the unknot
        gauss2 = [0]
        extend2 = [0]
        
    #Turn the original gauss code and the simplified gauss code into strings and return the resulting string
    gc1 = str(gauss1)[1:-1]+'/'+str(extend1)[1:-1]
    gc2 = str(gauss2)[1:-1]+':'+str(extend2)[1:-1]
    total_gauss = gc1+'*'+gc2
    
    return total_gauss